In [1]:
!pip install pykrx

Defaulting to user installation because normal site-packages is not writeable


In [2]:
from pykrx import stock
from pykrx import bond



In [3]:
import json

def get_market_ohlcv(start_date, end_date, ticker):
    stock_name = stock.get_market_ticker_name(ticker)
    df = stock.get_market_ohlcv(start_date, end_date, ticker)
    df['종목명'] = [stock_name] * len(df)

    return json.dumps(df.to_dict(orient='records'), ensure_ascii=False)

In [4]:
print(get_market_ohlcv("20220720", "20220810", "005930"))

[{"시가": 61800, "고가": 62100, "저가": 60500, "종가": 60500, "거래량": 16782238, "등락률": -0.6568144499178982, "종목명": "삼성전자"}, {"시가": 61100, "고가": 61900, "저가": 60700, "종가": 61800, "거래량": 12291374, "등락률": 2.1487603305785123, "종목명": "삼성전자"}, {"시가": 61800, "고가": 62200, "저가": 61200, "종가": 61300, "거래량": 10261310, "등락률": -0.8090614886731391, "종목명": "삼성전자"}, {"시가": 60900, "고가": 61900, "저가": 60800, "종가": 61100, "거래량": 9193681, "등락률": -0.3262642740619902, "종목명": "삼성전자"}, {"시가": 60800, "고가": 61900, "저가": 60800, "종가": 61700, "거래량": 6597211, "등락률": 0.9819967266775778, "종목명": "삼성전자"}, {"시가": 61300, "고가": 61900, "저가": 61200, "종가": 61800, "거래량": 7320997, "등락률": 0.1620745542949757, "종목명": "삼성전자"}, {"시가": 62300, "고가": 62600, "저가": 61600, "종가": 61900, "거래량": 10745302, "등락률": 0.16181229773462785, "종목명": "삼성전자"}, {"시가": 62400, "고가": 62600, "저가": 61300, "종가": 61400, "거래량": 15093120, "등락률": -0.8077544426494345, "종목명": "삼성전자"}, {"시가": 61000, "고가": 61700, "저가": 60300, "종가": 61300, "거래량": 13154816, "등락률": -0.1628664495114

In [5]:
tools = [
    {
        "type": "function",
        "function":
        {
            "name": "get_market_ohlcv",
            "description": "특정 종목에 대해 정해진 기간의 주가 데이터를 돌려줍니다.",
            "parameters": {
                "type": "object",
                "properties": {
                    "start_date": {
                        "type": "string",
                        "description": "YYYYMMDD 형식의 시작 날짜",
                    },
                    "end_date": {
                        "type": "string",
                        "description": "YYYYMMDD 형식의 종료 날짜",
                    },
                    "ticker": {
                        "type": "string",
                        "description": "6자리 숫자 형식의 종목 티커코드",
                    },
                },
                "required": ["start_date", "end_date", "ticker"],
            },
        }
    }
]


In [10]:
import os
import openai

openai.api_key = os.getenv("OPENAI_API_KEY")

messages= [
  {
    "role": "user",
    "content": "(주)에이피알(278470)의 2024년 12월부터 2025년 1월까지 주가를 분석해줘."
  }
]

response = openai.chat.completions.create(
  model="gpt-4o-mini",
  messages=messages,
  temperature=0,
  max_tokens=1024,
  tools=tools,
  tool_choice="auto" if tools is not None else None,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0
)

print(response.choices[0].message)

ChatCompletionMessage(content=None, refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=[ChatCompletionMessageToolCall(id='call_kDiL8WmIFjrfJKbml3hFRxg3', function=Function(arguments='{"start_date": "20241201", "end_date": "20250131", "ticker": "278470"}', name='get_market_ohlcv'), type='function')])


In [11]:
available_functions = {
    "get_market_ohlcv": get_market_ohlcv,
}

for tool_call in response.choices[0].message.tool_calls:
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(
        start_date=function_args['start_date'],
        end_date=function_args['end_date'],
        ticker=function_args['ticker']
    )
    messages.append(
        {
            "tool_call_id": tool_call.id,
            "role": "function",
            "name": function_name,
            "content": json.dumps(function_response, ensure_ascii=False)
        }
    )
    
second_response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)
print(second_response.choices[0].message.content)

에이피알(278470)의 2024년 12월부터 2025년 1월까지의 주가 데이터를 기반으로 분석해 보겠습니다. 주요 지표로는 시가, 고가, 저가, 종가, 거래량 및 등락률을 포함하였습니다. 

### 주가 분석 요약

#### 주요 가격 정보
- **2024년 12월~2025년 1월 주가 범위**:
  - **최고가**: 58400원
  - **최저가**: 46700원
  - **시작가**: 49250원
  - **종가**: 46200원

#### 가격 움직임
- **상승**: 두 번의 큰 상승이 있었으며, 한 번은 7.46%의 증가가 있었고, 다음으로 5.87%의 증가가 있었습니다.
- **하락**: 하락세를 보인 날도 있었으며, 특히 두 차례 모두 4% 이상의 하락폭이 있었습니다.
  
#### 거래량
- 우량한 거래량이 관찰되었으며, 하루 최대 거래량은 1,341,236주로 나타났습니다.
- 높은 거래량과 동시에 가격의 변동성이 높았던 시점이 있음을 보여주고 있습니다. 

#### 등락률(Now)
- 주가의 등락률은 변동성이 있었으며, 많은 일이 있던 시기임을 알 수 있습니다. 
- 가장 큰 등락률 변동은 -6.13%와 7.46%였습니다.

### 결론적인 분석
에이피알의 주가는 2024년 12월부터 2025년 1월까지 상당한 변동성을 보였고, 특히 높은 거래량을 기록했습니다. 이는 시장에서 투자자들의 주목을 받으며 매매가 활발히 이루어졌음을 의미합니다. 회사의 펀더멘털이나 시장 전반의 상황이 이 시기에 어떤 흐름이었는지에 따라, 향후 주가에 큰 영향을 미칠 것으로 보입니다. 

앞으로의 주가 전망을 위해서는 외부 경제 요소, 기업의 실적 발표 및 기타 시장의 동향을 주의 깊게 살펴봐야 할 것입니다.
